### QV to Ore

In [ ]:
import qiskit.ignis.verification.quantum_volume as qv

from qiskit2qore import qiskit_circuit_to_ore
from qiskit2ore.dump import dump_qiskit_circuit_assets
from chiron.dump import ore
from chiron.load.qore import load_qore
from chiron.dump import ore

import json
import subprocess
from pathlib import Path

# qv_circuits call requires connectivitiy to build circuits, will return QV circuits
# between min_qubits and max_qubits

# qiskit's qv_circuits algorithm will generate a batch of circuit for
# each width from min_qubits to max_qubits.
min_qubits=2
max_qubits=6
# number of circuits for each qubit width. QV circuits are random,
# and generally need many variants to approximate uniform unitaries.
max_circuits=1

print("converting qv to ore")

qubit_lists = [list(range(i)) for i in range(min_qubits, max_qubits + 1)]
_qv_circs, qv_circs_nomeas = qv.qv_circuits(qubit_lists, max_circuits)
for circuit_group in qv_circs_nomeas:
    for circuit in circuit_group:
        dump_qiskit_circuit_assets(circuit, preprocessor=True)

print("done")